<a href="https://colab.research.google.com/github/JADukeAIPI/AIPIProject/blob/main/WebScrapingAtlantaEvents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-search-results

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from serpapi import GoogleSearch
import json


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
today_date = datetime.today().strftime('%Y-%m-%d')

def get_end_date(number_days):

  end_date = datetime.today() + timedelta(days=number_days)
  end_date=end_date.strftime('%Y-%m-%d')
  return end_date

end_date = get_end_date(90)


In [5]:
def get_events(start_date = today_date, end_date = get_end_date(90)):
    '''
    Scrapes data from Creative Loafing, Google events, and Discover Atlanta for the next time period and organizes the data in a DataFrame

    Inputs:
        Date range in terms of strings of YYYY-MM-DD for today and future date; default is today to today + 90 days

    Returns:
        events_df(DataFrame): DataFrame containing the scraped data for all upcoming events
    '''
    
    # URLS to search  
    url = 'https://creativeloafing.com/atlanta-events/this-month' 
    url2 ='https://www.google.com/search?q=atlanta+convention+center+events&oq=atlanta+convention+center+events&aqs=chrome..69i57j0i512j0i22i30l6.4881j0j7&sourceid=chrome&ie=UTF-8&ibp=htl;events&rciv=evn&sa=X&ved=2ahUKEwjIsY_c4J_7AhX-lWoFHWBFBBcQ66QDKAZ6BAgIEAw#htichips=date:month&htischips=date;month&htidocid=L2F1dGhvcml0eS9ob3Jpem9uL2NsdXN0ZXJlZF9ldmVudC8yMDIyLTExLTExfDI1MTcwNDI2NDQ0NjM3MjA4ODg%3D&htivrt=events&fpstate=tldetail'
    url3 = 'https://discoveratlanta.com/events/all/?startdate='+today_date+'&enddate='+(end_date)+'&date_sort=this-month'
    url4 = 'https://www.google.com/search?lei=3P9qY9XjLZGvqtsPpuOckAE&q=atlanta+convention+calendar+2022&biw=1440&bih=764&dpr=2&ibp=htl;events&rciv=evn&sa=X&ved=2ahUKEwil94el95_7AhUZibAFHSDpDQkQ8eoFKAJ6BAgJEA8#htivrt=events&htidocid=L2F1dGhvcml0eS9ob3Jpem9uL2NsdXN0ZXJlZF9ldmVudC8yMDIyLTExLTExfDI1MTcwNDI2NDQ0NjM3MjA4ODg%3D&fpstate=tldetail'

    # Request the page and use BeautifulSoup to extract the contents
    page = requests.get(url)
    page2 = requests.get(url2)
    page3 = requests.get(url3)
    page4 = requests.get(url4)
    
    soup = BeautifulSoup(page.content, 'html.parser')
    soup2 = BeautifulSoup(page2.content, 'html.parser')
    soup3 = BeautifulSoup(page3.content, 'html.parser')
    soup4 = BeautifulSoup(page4.content, 'html.parser')

    #for now, just using url3
    events = soup3.find_all('article', class_='listing')
    
    event_dict={} #to store event info
    for event in events:
      title = event.find(attrs={'class':'listing-title'}).text
      dates = json.loads(event['data-eventdates'])
      location = event.find(attrs={'class':'subtitle location'}).text
      description = event.find(attrs={'class':"listing-copy detail"}).text     
      event_dict[title]={'Dates': dates, 'Location': location, 'Description': description}
 
    #create dataframe
    event_df = pd.DataFrame.from_dict(event_dict).T
    event_df.reset_index(inplace=True)
    event_df = event_df.rename(columns = {'index':'Event'})
    event_df = event_df.explode('Dates', ignore_index=True)
    event_df['Dates'] = pd.to_datetime(event_df['Dates'])
    event_df = event_df.sort_values(by='Dates', ascending=True).reset_index(drop=True)
    event_df = event_df[(event_df['Dates'] > start_date) & (event_df['Dates'] <= end_date)]
    return event_df

    raise NotImplementedError()

In [8]:
events_data_df = get_events('2022-11-20', '2023-02-03')

In [9]:
events_data_df

,Event,Dates,Location,Description
2081,Celebrate Thanksgiving All Week Long at Dantan...,2022-11-21,Dantanna’s Buckhead,Be extra thankful this year with Dantanna’s Th...
2082,Rudolph the Red-Nosed Reindeer,2022-11-21,Center For Puppetry Arts,A Holiday Classic Returns to the Stage Based o...
2083,IllumiNights at the Zoo,2022-11-21,Zoo Atlanta,
2084,PAW Patrol: Adventure Play,2022-11-21,Children’s Museum of Atlanta,Adventure Bay is coming to CMA! PAW Patrol: Ad...
2085,2022 Georgia Festival of Trees,2022-11-21,Georgia World Congress Center,Kick off your holiday season with the Georgia ...
...,...,...,...,...
3820,Stand-Up Comedy: Jim Gaffigan,2023-02-03,Cobb Energy Performing Arts Centre,Jim Gaffigan is a six-time Grammy nominated co...
3821,Deana Lawson Exhibit,2023-02-03,High Museum of Art,"Featuring work made over the past two decades,..."
3822,Stephen Burks Shelter in Place,2023-02-03,High Museum of Art,"In the wake of several global crises, designer..."
3823,Midtown Atlanta Food and Cocktail Tour,2023-02-03,Saint Mark United Methodist Church,Indulge in an eclectic array of cuisine & cock...
